In [31]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
from torch import Tensor

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #choose whether to use gpu or cpu


In [33]:
data_dir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder'  #path to the folder containing the images
# data_dir = r'C:\Users\Public\PartIIB project 2023_2024\PastData\helical_size'
# data_dir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder\second_half'
# from google.colab import drive
# drive.mount('/content/drive')
# data_dir = "/content/drive/My Drive/size_folder" #colab path
# data_dir = r'C:\Users\Chappyyyyyy\Documents\size_folder' #path to folder for chappy computer


In [34]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os

# dataset with cutsize 
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = sorted([f for f in os.listdir(root_dir) if os.path.isfile(os.path.join(root_dir, f))])
        self.labels = [self.extract_label(img) for img in self.images]
        self.cuttime = [self.extract_cuttime(img) for img in self.images]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.images[idx])
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        cuttime = self.cuttime[idx]
        return image, label, cuttime

    def extract_label(self, img_name):
        # Assuming that the label is part of the filename before the first underscore
        label = float(img_name[-17:-5]) #this is the right code
        # label = img_name
        return label
    
    def extract_cuttime(self, img_name):
        cuttime = float(img_name.split("t-")[1].split("_")[0])
        return cuttime

In [35]:
data_transform = transforms.Compose(
[transforms.ToTensor(),
transforms.Normalize((0.5,), (0.5,))]) 

custom_dataset = CustomImageDataset(root_dir=data_dir, transform=data_transform)

# # Accessing the data
# for img, label in custom_dataset:
#     print(f"Image shape: {img.shape}, Label: {label}")

print(len(custom_dataset))
# train_set, val_set, test_set = random_split(custom_dataset, [int(len(custom_dataset)*0.75), int(len(custom_dataset)*0.15), int(len(custom_dataset)*0.100056)]) #splits data into training, validation and test sets
train_set, test_set = random_split(custom_dataset, [int(len(custom_dataset)*0.75), int(len(custom_dataset)*0.25003)])
print(len(train_set))
# print(len(val_set))
print(len(test_set))

15800
11850
3950


In [36]:
#hyper parameters
num_epochs = 30
# num_epochs = 60
batch_size = 1
learning_rate = 0.0005
# learning_rate = 0.001

train = DataLoader(train_set, batch_size=batch_size, shuffle=True)
# val = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test = DataLoader(test_set, batch_size=batch_size, shuffle=False)

for (images, labels, cuttime) in train:
    print(labels.item())
    print(cuttime.item())
    break

for i in train:
    print(i)
    break

9.009037
92.0
[tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]]]), tensor([23.3589], dtype=torch.float64), tensor([73.], dtype=torch.float64)]


In [37]:
#residual block class
class ResidualBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels, 
            out_channels=out_channels, 
            kernel_size=(3, 3), 
            padding='same', 
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(
            in_channels=out_channels, 
            out_channels=out_channels, 
            kernel_size=(3, 3), 
            padding='same', 
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.downsample = None
        if in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(
                    in_channels, 
                    out_channels, 
                    kernel_size=(3, 3), 
                    padding='same', 
                    bias=False
                ),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x: Tensor) -> Tensor:
        identity = self.downsample(x) if self.downsample else x
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x + identity)

        return x

In [38]:
class ResNet34(nn.Module):
    def __init__(self):
        super(ResNet34,self).__init__()
        
        self.block1 = nn.Sequential(
            nn.Conv2d(1,64,kernel_size=2,stride=2,padding=3,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True)
        )
        
        self.block2 = nn.Sequential(
            nn.AvgPool2d(1,1),
            ResidualBlock(64,64),
            ResidualBlock(64,64),
            nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        )
        
        self.block3 = nn.Sequential(
            ResidualBlock(64,128),
            ResidualBlock(128,128),
            nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        )
        
        self.block4 = nn.Sequential(
            ResidualBlock(128,256),
            ResidualBlock(256,256),
            nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        )
        self.block5 = nn.Sequential(
            ResidualBlock(256,512),
            ResidualBlock(512,512),
            nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        )
        
        self.Avgpool = nn.AvgPool2d(2)
        # vowel_diacritic
        self.fc1 = nn.Linear(512+5,128)
        # grapheme_root
        self.fc2 = nn.Linear(128,64)
        # consonant_diacritic
        self.fc3 = nn.Linear(64,1)
        self.fc = nn.Linear(1,1)
        
    def forward(self,x, y):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.Avgpool(x)
        x = x.view(x.size(0),-1)
        y = self.fc(y)
        y = y.view(-1, 1).repeat(1,5)

        z = torch.cat((x,y), dim=1)
        z = F.relu(self.fc1(z))
        z = F.relu(self.fc2(z))
        z = self.fc3(z)
        return z

In [39]:
model = ResNet34().to(device)

# loss and optimizer
criterion = nn.MSELoss()
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.001)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.9)
#training loop
n_total_steps = len(train)
for epoch in range(num_epochs):
    total_loss = 0
    for i, (images, labels, cuttimes) in enumerate(train):
        images = images.to(device)
        labels = labels.to(device)
        cuttimes = cuttimes.to(device).float()

        #forward
        outputs = model(images, cuttimes)
        # print(labels)
        labels = labels.float() #this is the right code
        # labels = float(labels[0][-17:-5])
        loss = criterion(outputs, labels)

        #backward
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        # if (i+1) % 1000 ==0:
        # print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')
        total_loss += loss.item()
    print(f'epoch {epoch+1}/{num_epochs}, average loss = {total_loss/len(train):.4f}')
            

print("Finished Training")

epoch 1/30, average loss = 41.1992
epoch 2/30, average loss = 28.9726
epoch 3/30, average loss = 23.3563
epoch 4/30, average loss = 18.8641
epoch 5/30, average loss = 15.5626
epoch 6/30, average loss = 13.3323
epoch 7/30, average loss = 11.9692
epoch 8/30, average loss = 10.7381
epoch 9/30, average loss = 10.1678
epoch 10/30, average loss = 9.3967
epoch 11/30, average loss = 9.0805
epoch 12/30, average loss = 8.5740
epoch 13/30, average loss = 8.1461
epoch 14/30, average loss = 7.8296
epoch 15/30, average loss = 7.8006
epoch 16/30, average loss = 7.5963
epoch 17/30, average loss = 7.4060
epoch 18/30, average loss = 6.9936
epoch 19/30, average loss = 7.0832
epoch 20/30, average loss = 6.9160
epoch 21/30, average loss = 6.7785
epoch 22/30, average loss = 6.7006
epoch 23/30, average loss = 6.5159
epoch 24/30, average loss = 6.5294
epoch 25/30, average loss = 6.4719
epoch 26/30, average loss = 6.4583
epoch 27/30, average loss = 6.4900
epoch 28/30, average loss = 6.5577
epoch 29/30, average

In [40]:
with torch.no_grad(): # no need to calculate gradient
    squared_difference = 0
    for images, labels, cuttimes in test:
        images = images.to(device)
        labels = labels.to(device)
        cuttimes = cuttimes.to(device).float()
        outputs = model(images, cuttimes)
        predictions = float(outputs[0,0])

        squared_difference += (predictions - labels) ** 2
    
    rmse = torch.sqrt(squared_difference / len(test))
    print(f'RMSE = {rmse}')

RMSE = tensor([2.6635], dtype=torch.float64)


In [ ]:
#consideration
# 1. change the number of layers
# 2. use GELU instead of ReLU